In [1]:
! pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 873.0 kB/s eta 0:00:00


In [13]:
import tensorflow as tf
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from keras_tuner.engine.hyperparameters import HyperParameters

<ipython-input-13-3d035cb8c50d>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


# load dataset

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
x_train, x_test = x_train / 255.0, x_test / 255.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [26]:
def build_cnn_model(hp):
    model = keras.Sequential()
    inputsize = 28
    for i in range(hp.Int('num_conv_layers', min_value=1, max_value=5)):
      if i % 2 == 1:
        model.add(keras.layers.MaxPooling2D(pool_size=2, strides=2))
        inputsize = inputsize // 2

      model.add(keras.layers.Conv2D(
          filters=hp.Int('conv1_filters', min_value=1, max_value=256, step=16),
          kernel_size=3,
          activation='relu',
          input_shape=(inputsize, inputsize, 1)
      ))
      inputsize -= 1

    model.add(keras.layers.Flatten())

    for i in range(hp.Int('num_dense_layers', min_value=1, max_value=5)):
        model.add(keras.layers.Dense(
            units=hp.Int(f'dense_{i}_units', min_value=32, max_value=256, step=32),
            activation='relu'
        ))

    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [27]:
tuner = RandomSearch(
    build_cnn_model,
    objective='val_accuracy',
    max_trials=10,
    directory='RandomSearch_tuner',
    project_name='RandomSearch_tuner'
)

In [34]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


In [32]:
best_model = tuner.get_best_models(num_models=1)[0]

In [33]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 225)       2250      
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 225)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 225)       455850    
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 225)         455850    
                                                                 
 flatten (Flatten)           (None, 18225)             0         
                                                                 
 dense (Dense)               (None, 64)                1166464   
                                                        